# ElectroShop EDA: Investigate Session_ID Duplicates

Goals:
- Verify whether duplicate `Session_ID` counts are caused by nulls.
- Distinguish global vs within-day duplicates.
- Check for any true non-null duplicate IDs across or within days.
- Summarize root cause and suggested fixes.

Notes:
- Pandas `Series.duplicated()` treats repeated NaNs as duplicates after the first occurrence.
- We will compute duplicate counts both including and excluding null values.

In [2]:
# Imports and display options
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [9]:
# Load training data
root = Path.cwd().parent
train_path = root / "data" / "raw" / "dsba-m-1-challenge-purchase-prediction" / "train_dataset_M1_with_id.csv"
df = pd.read_csv(train_path)
print(f"Loaded {df.shape[0]:,} rows, {df.shape[1]} columns from {train_path.name}")
df.head(3)

Loaded 13,735 rows, 22 columns from train_dataset_M1_with_id.csv


,id,Age,Gender,Reviews_Read,Price,Discount,Category,Items_In_Cart,Time_of_Day,Email_Interaction,Device_Type,Payment_Method,Referral_Source,Socioeconomic_Status_Score,Engagement_Score,AB_Bucket,Price_Sine,PM_RS_Combo,Session_ID,Day,Campaign_Period,Purchase
0,1,NaN,1.0,3.0,592.975,22.0,1.0,6.0,afterno0n,0.0,Mobile,Credit,NaN,7.26,1.856520,3.0,0.999047,Credit:Social_media,S0000003,59,False,0
1,2,25.0,1.0,1.0,511.279,12.0,0.0,3.0,morning,1.0,Tablet,Cash,Social_media,8.30,1.868138,5.0,-0.129689,Cash:Social_media,S0000005,29,True,1
2,3,22.0,0.0,3.0,218.360,2.0,1.0,4.0,evening,1.0,Mobile,Bank,Social_media,6.61,1.223445,0.0,-0.421646,Bank:Social_media,S0000007,16,False,0


In [16]:
# Global duplicate analysis for Session_ID
s = df['Session_ID']
n_null = s.isna().sum()
dup_all = s.duplicated().sum()
dup_nonnull = s[s.notna()].duplicated().sum()
print("Session_ID nulls:", int(n_null))
print("Duplicates (including nulls):", int(dup_all))
print("Duplicates (excluding nulls):", int(dup_nonnull))
print("Are there any true duplicate non-null Session_IDs?", "YES" if dup_nonnull > 0 else "NO")
predicted_from_nulls = max(n_null - 1, 0)
print("Do nulls alone explain duplicate count?", dup_all - dup_nonnull == predicted_from_nulls)

if dup_nonnull > 0:
    dupe_ids = s[s.notna()][s[s.notna()].duplicated(keep=False)].unique()
    display(df[df['Session_ID'].isin(dupe_ids)].sort_values(['Session_ID','Day']).head(10))


# Show rows where Session_ID is null
null_rows = df[df['Session_ID'].isna()]
print(f"Total rows with null Session_ID: {len(null_rows)}")
display(null_rows.head(20))
# also show a random sample for variety
display(null_rows.sample(10, random_state=42))


Session_ID nulls: 280
Duplicates (including nulls): 279
Duplicates (excluding nulls): 0
Are there any true duplicate non-null Session_IDs? NO
Do nulls alone explain duplicate count? True
Total rows with null Session_ID: 280


,id,Age,Gender,Reviews_Read,Price,Discount,Category,Items_In_Cart,Time_of_Day,Email_Interaction,Device_Type,Payment_Method,Referral_Source,Socioeconomic_Status_Score,Engagement_Score,AB_Bucket,Price_Sine,PM_RS_Combo,Session_ID,Day,Campaign_Period,Purchase
160,161,57.0,1.0,1.0,34.462,45.0,1.0,2.0,morning,0.0,Mobile,Cash,Social_media,2.72,5.784830,6.0,0.941494,Cash:Social_media,NaN,63,False,1
177,178,18.0,1.0,1.0,640.451,39.0,3.0,5.0,evening,1.0,Mobile,Credit,NaN,3.69,0.431621,1.0,-0.853722,Credit:Search_engine,NaN,26,True,0
214,215,23.0,0.0,1.0,268.950,19.0,1.0,6.0,morning,0.0,Desktop,NaN,Email,3.32,0.609066,2.0,0.693472,Bank:Email,NaN,61,False,0
234,235,25.0,1.0,4.0,574.440,6.0,2.0,3.0,morning,0.0,Tablet,Credit,Direct,8.39,2.273156,2.0,0.477148,Credit:Direct,NaN,19,False,1
309,310,27.0,1.0,2.0,785.826,11.0,1.0,5.0,evening,1.0,Desktop,Cash,Email,0.00,1.957085,0.0,-0.319590,Cash:Email,NaN,60,False,0
429,430,35.0,0.0,1.0,444.002,7.0,2.0,5.0,morning,0.0,Desktop,Bank,Ads,2.64,0.502176,5.0,-0.449284,Bank:Ads,NaN,49,True,0
606,607,NaN,1.0,2.0,502.256,10.0,3.0,3.0,afternoon,1.0,Desktop,Bank,NaN,15.39,0.205667,6.0,0.362755,Bank:Ads,NaN,4,False,0
705,706,43.0,1.0,4.0,844.024,10.0,0.0,2.0,evening,0.0,Desktop,Cash,Search_engine,2.16,0.544855,5.0,0.231547,Cash:Search_engine,NaN,1,False,1
747,748,20.0,0.0,3.0,322.440,9.0,3.0,9.0,afternoon,1.0,Mobile,NaN,Social_media,6.28,0.524476,6.0,-0.805324,Cash:Social_media,NaN,67,False,0
819,820,40.0,1.0,0.0,562.135,44.0,4.0,0.0,evening,0.0,Mobile,Cash,NaN,6.44,4.207375,5.0,-0.185130,Cash:Direct,NaN,49,True,1


,id,Age,Gender,Reviews_Read,Price,Discount,Category,Items_In_Cart,Time_of_Day,Email_Interaction,Device_Type,Payment_Method,Referral_Source,Socioeconomic_Status_Score,Engagement_Score,AB_Bucket,Price_Sine,PM_RS_Combo,Session_ID,Day,Campaign_Period,Purchase
2285,2286,19.0,0.0,1.0,586.6040,44.0,0.0,3.0,morning,0.0,Mobile,Cash,Ads,10.77,2.754694,1.0,0.921988,Cash:Ads,NaN,32,True,0
5490,5491,42.0,1.0,3.0,429.7220,7.0,0.0,2.0,evening,NaN,Mobile,Bank,NaN,4.02,3.193672,5.0,-0.951875,Bank:Email,NaN,69,False,0
11924,11925,35.0,0.0,4.0,NaN,80.0,2.0,2.0,morning,1.0,Desktop,Bank,Social_media,7.11,1.282475,2.0,-0.472080,Bank:Social_media,NaN,39,True,0
12870,12871,35.0,1.0,6.0,634.1440,4.0,3.0,0.0,evening,0.0,Desktop,Cash,Search_engine,3.40,0.114988,5.0,-0.623102,Cash:Search_engine,NaN,38,True,1
7521,7522,25.0,0.0,6.0,6136.9289,25.0,2.0,3.0,morning,1.0,Mobile,Bank,Search_engine,8.44,1.222927,3.0,-0.180360,Bank:Search_engine,NaN,23,False,0
819,820,40.0,1.0,0.0,562.1350,44.0,4.0,0.0,evening,0.0,Mobile,Cash,NaN,6.44,4.207375,5.0,-0.185130,Cash:Direct,NaN,49,True,1
7248,7249,33.0,1.0,4.0,639.8440,37.0,1.0,3.0,evening,NaN,NaN,PayPal,Email,0.67,2.924844,4.0,-0.835680,NaN,NaN,67,NaN,0
9827,9828,28.0,0.0,3.0,107.6570,14.0,2.0,1.0,morning,0.0,Mobile,NaN,Search_engine,11.37,0.131953,2.0,-0.297660,Credit:Search_engine,NaN,36,True,1
7191,7192,28.0,0.0,7.0,984.2080,40.0,0.0,3.0,evening,0.0,Desktop,NaN,Direct,9.81,3.041962,0.0,-0.955440,NaN,NaN,20,False,0
7540,7541,35.0,0.0,5.0,305.7720,21.0,1.0,6.0,afternoon,0.0,Mobile,cash,Social_media,2.29,4.414747,3.0,-0.957835,Bank:Social_media,NaN,29,True,0


In [4]:
# Within-day duplicates excluding nulls
within_day_dupes = (df[df['Session_ID'].notna()]
                     .groupby('Day')['Session_ID']
                     .apply(lambda x: x.duplicated().sum()))
nonzero_days = within_day_dupes[within_day_dupes > 0].sort_values(ascending=False)
print("Total within-day duplicates (excluding nulls):", int(within_day_dupes.sum()))
print("Days with within-day duplicates (excluding nulls):", len(nonzero_days))
display(nonzero_days.head(20))

Total within-day duplicates (excluding nulls): 0
Days with within-day duplicates (excluding nulls): 0


Series([], Name: Session_ID, dtype: int64)

In [11]:
# Distribution of null Session_ID across days
null_by_day = (df['Session_ID'].isna()
               .groupby(df['Day'])
               .sum()
               .sort_values(ascending=False))
print("Null Session_ID by day (top 20):")
display(null_by_day.head(20))

Null Session_ID by day (top 20):


Day
67    8
5     8
40    8
1     7
69    7
34    7
28    7
58    7
39    6
55    6
13    6
11    6
7     6
21    5
30    5
32    5
33    5
27    5
49    5
22    5
Name: Session_ID, dtype: int64

In [12]:
# Cross-day duplicates among non-null Session_IDs
counts = (df[df['Session_ID'].notna()]
           .groupby('Session_ID')
           .size())
multi = counts[counts > 1]
print("Non-null Session_IDs appearing multiple times:", int(len(multi)))
if len(multi) > 0:
    sample_id = multi.index[0]
    display(df[df['Session_ID'] == sample_id].sort_values('Day')[['id','Session_ID','Day']].head(10))

Non-null Session_IDs appearing multiple times: 0


In [13]:
# Whitespace/format sanity check on Session_ID
s_orig = df['Session_ID']
s_stripped = s_orig.astype('string').str.strip()
changed = ((s_stripped != s_orig.astype('string')) & s_orig.notna()).sum()
print("Non-null Session_ID values that change after strip():", int(changed))
dup_nonnull_stripped = s_stripped[s_stripped.notna()].duplicated().sum()
print("Duplicates (excluding nulls) after strip():", int(dup_nonnull_stripped))

Non-null Session_ID values that change after strip(): 0
Duplicates (excluding nulls) after strip(): 0


## Conclusion

- The global duplicate count for `Session_ID` is fully explained by repeated nulls (NaNs).
- When excluding nulls, duplicate `Session_ID` counts should be zero both globally and within each day.
- Action items:
  - Treat rows with null `Session_ID` as invalid per data contract (drop or impute upstream).
  - Keep `id` as the canonical unique key; do not rely on `Session_ID` where nulls exist.
  - Ensure ingestion enforces non-null `Session_ID` going forward.

In [55]:
# Load raw data
root = Path.cwd().parent
train_path = root / "data" / "raw" / "dsba-m-1-challenge-purchase-prediction" / "train_dataset_M1_with_id.csv"
df = pd.read_csv(train_path)

In [56]:
# Record before true count
before_recompute = int(df['Campaign_Period'].sum())

# recompute Campaign_Period per definition: Day in [25,50] or [75,90]
df['Campaign_Period'] = df['Day'].between(25, 50) | df['Day'].between(75, 90)
df['Campaign_Period'] = df['Campaign_Period'].astype(bool)

# sanity check
after_recompute = int(df['Campaign_Period'].sum())
print("Before vs after", before_recompute, after_recompute)

Before vs after 4953 5048


In [57]:
# Drop rows with null Session_ID (potentially impute surrogate Session_IDs later)
before = len(df)
df = df[df['Session_ID'].notna()].copy()
df['Session_ID'] = df['Session_ID'].astype('string').str.strip()
after = len(df)

print(f"Dropped {before - after} rows with null Session_ID.")
assert df['Session_ID'].notna().all()
assert not df['Session_ID'].duplicated().any(), "Session_ID must be globally unique."



Dropped 280 rows with null Session_ID.


In [58]:
# Clean Time_of_Day: case-insensitive standardization and fix '0'->'o'
orig = df['Time_of_Day'].astype('string')
normalized = orig.str.strip().str.lower().str.replace('0', 'o', regex=False)
allowed = {'morning', 'afternoon', 'evening'}

# # display the values of Time_of_Day that are still invalid after normalization
# invalid_values = normalized[~normalized.isin(allowed) & normalized.notna()].unique()
# print("Unrecognized Time_of_Day values after normalization:", invalid_values)

# # display unique values before and after normalization
# print("Unique Time_of_Day values before normalization:", orig.unique())
# print("Unique Time_of_Day values after normalization:", normalized.unique())

# how many values changed due to normalization (case/zeros)
changed_mask = orig.notna() & (orig != normalized)
n_changed = int(changed_mask.sum())


# keep only allowed values, set others to <NA>
normalized = normalized.where(normalized.isin(allowed))
n_invalid_remaining = int(normalized.isna().sum())

df['Time_of_Day'] = normalized

print(f"Time_of_Day: standardized {n_changed} values; {n_invalid_remaining} rows set to NaN/unrecognized.")
assert df['Time_of_Day'].dropna().isin(allowed).all()



Time_of_Day: standardized 92 values; 271 rows set to NaN/unrecognized.


In [59]:
# print range of Day
print("Day range:", df['Day'].min(), "to", df['Day'].max())

Day range: 1 to 70


In [60]:
# Convert string categorical columns to category dtype for better memory efficiency
categorical_string_cols = ['Time_of_Day', 'Device_Type']
for col in categorical_string_cols:
    if col in df.columns:
        # Convert to category dtype (keeps NaN as NaN)
        df[col] = df[col].astype('category')
        print(f"{col}: converted to category dtype with {df[col].cat.categories.tolist()} categories")

# Verify the change
print(f"\nTime_of_Day dtype: {df['Time_of_Day'].dtype}")
print(f"Device_Type dtype: {df['Device_Type'].dtype}")

Time_of_Day: converted to category dtype with ['afternoon', 'evening', 'morning'] categories
Device_Type: converted to category dtype with ['Desktop', 'Mobile', 'Tablet'] categories

Time_of_Day dtype: category
Device_Type dtype: category


In [61]:
# Check PM_RS_Combo structure and extract Payment_Method and Referral_Source for null values
print("=" * 80)
print("ANALYZING PM_RS_Combo FOR MISSING VALUE IMPUTATION")
print("=" * 80)

# Check current null counts
pm_null_before = df['Payment_Method'].isna().sum()
rs_null_before = df['Referral_Source'].isna().sum()
print(f"\n📊 Current null counts:")
print(f"  Payment_Method: {pm_null_before:,}")
print(f"  Referral_Source: {rs_null_before:,}")

# Examine PM_RS_Combo structure
print(f"\n🔍 PM_RS_Combo sample values:")
print(df['PM_RS_Combo'].dropna().head(10).tolist())

# Check if PM_RS_Combo has pattern like "Credit:Social_media"
sample_combo = df['PM_RS_Combo'].dropna().iloc[0] if df['PM_RS_Combo'].notna().any() else None
print(f"\n📝 Example PM_RS_Combo: {sample_combo}")

# Count rows where Payment_Method is null but PM_RS_Combo is not
pm_null_with_combo = df[df['Payment_Method'].isna() & df['PM_RS_Combo'].notna()]
rs_null_with_combo = df[df['Referral_Source'].isna() & df['PM_RS_Combo'].notna()]

print(f"\n✅ Rows where we can potentially fill from PM_RS_Combo:")
print(f"  Payment_Method nulls with PM_RS_Combo: {len(pm_null_with_combo):,}")
print(f"  Referral_Source nulls with PM_RS_Combo: {len(rs_null_with_combo):,}")

# Split PM_RS_Combo and extract Payment_Method and Referral_Source
print(f"\n🔧 Extracting values from PM_RS_Combo...")

# Create temporary columns with extracted values
df['PM_from_combo'] = df['PM_RS_Combo'].astype('string').str.split(':', expand=True)[0]
df['RS_from_combo'] = df['PM_RS_Combo'].astype('string').str.split(':', expand=True)[1]

# Show some examples of the extraction
print(f"\n📋 Extraction examples:")
display(df[['PM_RS_Combo', 'PM_from_combo', 'RS_from_combo']].dropna(subset=['PM_RS_Combo']).head(10))

# Fill nulls in Payment_Method with PM_from_combo
df['Payment_Method'] = df['Payment_Method'].fillna(df['PM_from_combo'])

# Fill nulls in Referral_Source with RS_from_combo
df['Referral_Source'] = df['Referral_Source'].fillna(df['RS_from_combo'])

# Check new null counts
pm_null_after = df['Payment_Method'].isna().sum()
rs_null_after = df['Referral_Source'].isna().sum()

print(f"\n📊 Null counts after filling from PM_RS_Combo:")
print(f"  Payment_Method: {pm_null_before:,} → {pm_null_after:,} (filled {pm_null_before - pm_null_after:,})")
print(f"  Referral_Source: {rs_null_before:,} → {rs_null_after:,} (filled {rs_null_before - rs_null_after:,})")

# Clean up temporary columns
df = df.drop(columns=['PM_from_combo', 'RS_from_combo'])

# Show final value counts
print(f"\n📈 Final value distributions:")
print(f"\nPayment_Method:")
print(df['Payment_Method'].value_counts(dropna=False))
print(f"\nReferral_Source:")
print(df['Referral_Source'].value_counts(dropna=False))

print("=" * 80)


ANALYZING PM_RS_Combo FOR MISSING VALUE IMPUTATION

📊 Current null counts:
  Payment_Method: 2,009
  Referral_Source: 1,973

🔍 PM_RS_Combo sample values:
['Credit:Social_media', 'Cash:Social_media', 'Bank:Social_media', 'Cash:Social_media', 'Bank:Direct', 'Bank:Search_engine', 'Bank:Search_engine', 'PayPal:Search_engine', 'Bank:Social_media', 'Bank:Social_media']

📝 Example PM_RS_Combo: Credit:Social_media

✅ Rows where we can potentially fill from PM_RS_Combo:
  Payment_Method nulls with PM_RS_Combo: 1,968
  Referral_Source nulls with PM_RS_Combo: 1,932

🔧 Extracting values from PM_RS_Combo...

📋 Extraction examples:


,PM_RS_Combo,PM_from_combo,RS_from_combo
0,Credit:Social_media,Credit,Social_media
1,Cash:Social_media,Cash,Social_media
2,Bank:Social_media,Bank,Social_media
4,Cash:Social_media,Cash,Social_media
5,Bank:Direct,Bank,Direct
6,Bank:Search_engine,Bank,Search_engine
7,Bank:Search_engine,Bank,Search_engine
8,PayPal:Search_engine,PayPal,Search_engine
9,Bank:Social_media,Bank,Social_media
10,Bank:Social_media,Bank,Social_media



📊 Null counts after filling from PM_RS_Combo:
  Payment_Method: 2,009 → 41 (filled 1,968)
  Referral_Source: 1,973 → 41 (filled 1,932)

📈 Final value distributions:

Payment_Method:
Payment_Method
Bank             3324
PayPal           3314
Cash             3270
Credit           3106
pay_pal            56
creDit             53
PayPAl             52
cash               51
pay pal            51
bank_transfer      48
CASH               48
<NA>               41
BAnk                6
CredIt              5
CaSh                4
CRedit              4
BaNk                4
CreDit              3
BanK                3
CrediT              2
CrEdit              2
CAsh                1
PayPaL              1
CRediT              1
PaYPal              1
CasH                1
PAyPal              1
creDIt              1
CRedIt              1
Name: count, dtype: int64

Referral_Source:
Referral_Source
Search_engine    2666
Social_media     2630
Direct           2626
Ads              2611
Email           

In [62]:
# Normalize Payment_Method typos (lowercase + regex) and map to canonical categories.

# create cleaned lowercase token (remove punctuation/whitespace)
pm = df['Payment_Method'].astype('string').str.lower().str.strip()
pm_clean = pm.str.replace(r'[^a-z0-9]+', '', regex=True)

# map common typo patterns to canonical labels
conds = [
    pm_clean.str.contains(r'pay.*pal', na=False),
    pm_clean.str.contains(r'cre.*it|^cred', na=False),
    pm_clean.str.contains(r'cas.*h|^cash', na=False),
    pm_clean.str.contains(r'bank', na=False)
]
choices = ['PayPal', 'Credit', 'Cash', 'Bank']

# Use 'Unknown' as default instead of np.nan to avoid dtype conflict
df['Payment_Method_norm'] = np.select(conds, choices, default='Unknown')
# Set 'Unknown' back to NaN where original was NaN
df.loc[df['Payment_Method'].isna(), 'Payment_Method_norm'] = pd.NA

# quick sanity check
df['Payment_Method_norm'].value_counts(dropna=False)

# replace Payment_Method with Payment_Method_norm
df['Payment_Method'] = df['Payment_Method_norm']

# drop Payment_Method_norm
df = df.drop(columns=['Payment_Method_norm'])


In [63]:
# check if Payment_Method has nulls
null_count = df['Payment_Method'].isna().sum()
print(f"Payment_Method nulls after normalization: {null_count}")

Payment_Method nulls after normalization: 41


In [64]:
# Normalize Referral_Source: lowercase, strip whitespace, fix '0'->'o'
rs = df['Referral_Source'].astype('string').str.lower().str.strip()
rs_clean = rs.str.replace('0', 'o', regex=False).str.replace('-', '_', regex=False).str.replace(' ', '', regex=False)

# Map to canonical labels
conds = [
    rs_clean.str.contains(r'social.*media', na=False),
    rs_clean.str.contains(r'direct', na=False),
    rs_clean.str.contains(r'search.*engine', na=False),
    rs_clean.str.contains(r'^ads?$|^ad$', na=False),
    rs_clean.str.contains(r'email', na=False)
]
choices = ['Social_media', 'Direct', 'Search_engine', 'Ads', 'Email']

# Use 'Unknown' as default, then set back to NaN where original was NaN
df['Referral_Source_norm'] = np.select(conds, choices, default='Unknown')
df.loc[df['Referral_Source'].isna(), 'Referral_Source_norm'] = pd.NA

# Replace original with normalized version
df['Referral_Source'] = df['Referral_Source_norm']
df = df.drop(columns=['Referral_Source_norm'])

# Verify standardization
print("Standardized Referral_Source values:")
print(df['Referral_Source'].value_counts(dropna=False))

Standardized Referral_Source values:
Referral_Source
Search_engine    2729
Direct           2685
Social_media     2682
Email            2666
Ads              2652
<NA>               41
Name: count, dtype: int64


In [65]:
# print unique values of categorical columns
categorical_cols = ['Gender', 'Payment_Method', 'Referral_Source', 'Time_of_Day', 'Device_Type']
for col in categorical_cols:
    unique_values = df[col].dropna().unique()
    print(f"Unique values in {col}: {unique_values}")

Unique values in Gender: [1. 0.]
Unique values in Payment_Method: ['Credit' 'Cash' 'Bank' 'PayPal']
Unique values in Referral_Source: ['Social_media' 'Direct' 'Search_engine' 'Ads' 'Email']
Unique values in Time_of_Day: ['afternoon', 'morning', 'evening']
Categories (3, string): [afternoon, evening, morning]
Unique values in Device_Type: ['Mobile', 'Tablet', 'Desktop']
Categories (3, object): ['Desktop', 'Mobile', 'Tablet']


In [53]:
# Save imputed dataset
imputed_path = root / "data" / "interim" / "train_dataset_M1_interim.csv"
df.to_csv(imputed_path, index=False)
print(f"Saved imputed data: {df.shape[0]:,} rows, {df.shape[1]} columns")
print(f"New features added: {df.shape[1] - 22} missing indicators")

Saved imputed data: 13,455 rows, 22 columns
New features added: 0 missing indicators


In [69]:
# Investigate Price_Sine - check metadata and potential hash-based calculation
print("=" * 80)
print("ANALYZING Price_Sine CALCULATION")
print("=" * 80)

# List all columns in the dataset
print(f"\n📋 All columns in dataset:")
print(df.columns.tolist())

# Check if Price_Sine is documented in metadata
metadata_columns = [
    'Purchase', 'Age', 'Gender', 'Reviews_Read', 'Price', 'Discount', 
    'Category', 'Items_In_Cart', 'Time_of_Day', 'Email_Interaction',
    'Device_Type', 'Payment_Method', 'Referral_Source', 
    'Socioeconomic_Status_Score', 'Engagement_Score', 
    'Session_ID', 'Day', 'Campaign_Period'
]

extra_columns = [col for col in df.columns if col not in metadata_columns and col != 'id']
print(f"\n⚠️  Columns NOT in metadata: {extra_columns}")

if 'Price_Sine' in df.columns:
    print(f"\n🔍 Price_Sine detailed analysis:")
    print(f"  Null count: {df['Price_Sine'].isna().sum()}")
    print(f"  Unique values: {df['Price_Sine'].nunique()}")
    print(f"  Range: [{df['Price_Sine'].min():.6f}, {df['Price_Sine'].max():.6f}]")
    
    # Check if nulls in Price_Sine correspond to nulls in Price
    print(f"\n🔗 Relationship with Price nulls:")
    price_null = df['Price'].isna()
    price_sine_null = df['Price_Sine'].isna()
    print(f"  Price nulls: {price_null.sum()}")
    print(f"  Price_Sine nulls: {price_sine_null.sum()}")
    print(f"  Both null: {(price_null & price_sine_null).sum()}")
    print(f"  Only Price_Sine null: {(~price_null & price_sine_null).sum()}")
    
    # Test if it could be based on Session_ID (hash-like)
    print(f"\n🔍 Testing Session_ID-based calculation:")
    # For each unique Session_ID, check if Price_Sine is constant
    session_sine = df[df['Price_Sine'].notna()].groupby('Session_ID')['Price_Sine'].nunique()
    all_unique = (session_sine == 1).all()
    print(f"  Each Session_ID has unique Price_Sine: {all_unique}")
    
    if all_unique:
        print(f"  🎯 Price_Sine appears to be SESSION-SPECIFIC (one value per session)")
        # Try to find the pattern
        # Could be: sin(hash(Session_ID))
        print(f"\n  Testing: sin(hash(Session_ID) % scale)...")
        
        # Show sample mapping
        print(f"\n  📋 Sample Session_ID -> Price_Sine mapping:")
        sample = df[['Session_ID', 'Price_Sine']].dropna().drop_duplicates('Session_ID').head(10)
        display(sample)
    
    # Test if it's derived from id
    print(f"\n🔍 Testing if based on row id:")
    mask = df['Price_Sine'].notna()
    df_test = df[mask].copy()
    
    # Try sin(id * constant)
    for scale in [0.001, 0.01, 0.1, 1, 10]:
        test_val = np.sin(df_test['id'] * scale)
        if np.allclose(df_test['Price_Sine'], test_val, rtol=1e-5):
            print(f"  ✅ MATCH: Price_Sine = sin(id * {scale})")
            break
    else:
        print(f"  ❌ Not a simple function of id")
    
    # Final hypothesis
    print(f"\n💡 CONCLUSION:")
    print(f"  Price_Sine is likely a synthetic/engineered feature not documented in metadata.")
    print(f"  It appears to be session-specific (one value per Session_ID).")
    print(f"  Possible purposes:")
    print(f"    - Noise injection for data augmentation")
    print(f"    - Hash-based feature for model robustness testing")
    print(f"    - Cyclic encoding of some hidden session property")

print("=" * 80)

ANALYZING Price_Sine CALCULATION

📋 All columns in dataset:
['id', 'Age', 'Gender', 'Reviews_Read', 'Price', 'Discount', 'Category', 'Items_In_Cart', 'Time_of_Day', 'Email_Interaction', 'Device_Type', 'Payment_Method', 'Referral_Source', 'Socioeconomic_Status_Score', 'Engagement_Score', 'AB_Bucket', 'Price_Sine', 'PM_RS_Combo', 'Session_ID', 'Day', 'Campaign_Period', 'Purchase', 'test_sin_price']

⚠️  Columns NOT in metadata: ['AB_Bucket', 'Price_Sine', 'PM_RS_Combo', 'test_sin_price']

🔍 Price_Sine detailed analysis:
  Null count: 279
  Unique values: 13088
  Range: [-1.000000, 1.000000]

🔗 Relationship with Price nulls:
  Price nulls: 616
  Price_Sine nulls: 279
  Both null: 11
  Only Price_Sine null: 268

🔍 Testing Session_ID-based calculation:
  Each Session_ID has unique Price_Sine: True
  🎯 Price_Sine appears to be SESSION-SPECIFIC (one value per session)

  Testing: sin(hash(Session_ID) % scale)...

  📋 Sample Session_ID -> Price_Sine mapping:


,Session_ID,Price_Sine
0,S0000003,0.999047
1,S0000005,-0.129689
2,S0000007,-0.421646
3,S0000008,-0.988239
4,S0000009,0.695737
5,S0000010,-0.874546
6,S0000011,0.998348
7,S0000013,0.862095
8,S0000014,0.804534
9,S0000016,0.448290



🔍 Testing if based on row id:
  ❌ Not a simple function of id

💡 CONCLUSION:
  Price_Sine is likely a synthetic/engineered feature not documented in metadata.
  It appears to be session-specific (one value per Session_ID).
  Possible purposes:
    - Noise injection for data augmentation
    - Hash-based feature for model robustness testing
    - Cyclic encoding of some hidden session property


## Convert to float to int needs to be done while in df as csv does not support it

In [52]:
# Convert float columns to nullable integer types (Int64 supports NaN)
print("=" * 80)
print("FLOAT TO INTEGER CONVERSION")
print("=" * 80)

# Columns that should be integers but are currently float
float_to_int_cols = {
    'Age': 'Int64',           # nullable int (has NaN)
    'Gender': 'Int8',         # 0/1 binary (has NaN)
    'Reviews_Read': 'Int64',  # count (has NaN)
    'Discount': 'Int8',       # 0-100 percentage (has NaN)
    'Category': 'Int8',       # 0-4 categories (has NaN)
    'Items_In_Cart': 'Int64', # count (has NaN)
    'Email_Interaction': 'Int8',  # 0/1 binary (has NaN)
    'AB_Bucket': 'Int8'       # categorical (has NaN)
}

print("\n🔍 Pre-conversion Analysis:")
print("-" * 80)

for col, target_dtype in float_to_int_cols.items():
    if col not in df.columns:
        print(f"⚠️  {col}: NOT FOUND IN DATAFRAME")
        continue
    
    # Check current state
    current_dtype = df[col].dtype
    n_null = df[col].isna().sum()
    n_total = len(df)
    
    # Check if values are actually integers (ignoring NaN)
    non_null_values = df[col].dropna()
    is_integer_like = (non_null_values == non_null_values.astype(int)).all() if len(non_null_values) > 0 else True
    
    # Check for fractional values
    has_decimals = False
    if len(non_null_values) > 0 and is_integer_like:
        # Double-check by looking at modulo
        has_decimals = (non_null_values % 1 != 0).any()
    
    print(f"\n{col}:")
    print(f"  Current dtype: {current_dtype}")
    print(f"  Null count: {n_null:,} ({n_null/n_total*100:.1f}%)")
    print(f"  Target dtype: {target_dtype}")
    
    if has_decimals:
        print(f"  ❌ UNSUITABLE: Contains fractional values!")
        # Show some examples
        fractional = non_null_values[non_null_values % 1 != 0].head()
        print(f"  Examples: {fractional.values}")
    elif not is_integer_like:
        print(f"  ❌ UNSUITABLE: Values cannot be safely converted to integers")
        print(f"  Value range: [{non_null_values.min()}, {non_null_values.max()}]")
    else:
        print(f"  ✅ SUITABLE for conversion")
        if len(non_null_values) > 0:
            print(f"  Value range: [{int(non_null_values.min())}, {int(non_null_values.max())}]")

print("\n" + "=" * 80)
print("PERFORMING CONVERSION")
print("=" * 80)

conversion_results = {'success': [], 'failed': [], 'skipped': []}

for col, target_dtype in float_to_int_cols.items():
    if col not in df.columns:
        conversion_results['skipped'].append((col, 'Column not found'))
        continue
    
    try:
        # Check if conversion is safe
        non_null_values = df[col].dropna()
        
        if len(non_null_values) > 0:
            # Check for fractional values
            if (non_null_values % 1 != 0).any():
                conversion_results['failed'].append((col, 'Contains fractional values'))
                print(f"❌ {col}: SKIPPED (contains decimals)")
                continue
        
        # Perform conversion using pandas nullable integer type
        df[col] = df[col].astype(target_dtype)
        conversion_results['success'].append(col)
        print(f"✅ {col}: {df[col].dtype} (nulls preserved: {df[col].isna().sum():,})")
        
    except Exception as e:
        conversion_results['failed'].append((col, str(e)))
        print(f"❌ {col}: FAILED - {e}")

print("\n" + "=" * 80)
print("CONVERSION SUMMARY")
print("=" * 80)
print(f"✅ Successfully converted: {len(conversion_results['success'])} columns")
for col in conversion_results['success']:
    print(f"   - {col}")

if conversion_results['failed']:
    print(f"\n❌ Failed conversions: {len(conversion_results['failed'])} columns")
    for col, reason in conversion_results['failed']:
        print(f"   - {col}: {reason}")

if conversion_results['skipped']:
    print(f"\n⚠️  Skipped: {len(conversion_results['skipped'])} columns")
    for col, reason in conversion_results['skipped']:
        print(f"   - {col}: {reason}")

print("\n" + "=" * 80)
print("FINAL DTYPES VERIFICATION")
print("=" * 80)
for col in float_to_int_cols.keys():
    if col in df.columns:
        print(f"{col:30s} → {str(df[col].dtype):15s} (nulls: {df[col].isna().sum():,})")

print("=" * 80)

FLOAT TO INTEGER CONVERSION

🔍 Pre-conversion Analysis:
--------------------------------------------------------------------------------

Age:
  Current dtype: float64
  Null count: 2,036 (15.1%)
  Target dtype: Int64
  ✅ SUITABLE for conversion
  Value range: [18, 65]

Gender:
  Current dtype: float64
  Null count: 256 (1.9%)
  Target dtype: Int8
  ✅ SUITABLE for conversion
  Value range: [0, 1]

Reviews_Read:
  Current dtype: float64
  Null count: 287 (2.1%)
  Target dtype: Int64
  ✅ SUITABLE for conversion
  Value range: [0, 11]

Discount:
  Current dtype: float64
  Null count: 271 (2.0%)
  Target dtype: Int8
  ✅ SUITABLE for conversion
  Value range: [0, 90]

Category:
  Current dtype: float64
  Null count: 280 (2.1%)
  Target dtype: Int8
  ✅ SUITABLE for conversion
  Value range: [0, 4]

Items_In_Cart:
  Current dtype: float64
  Null count: 279 (2.1%)
  Target dtype: Int64
  ✅ SUITABLE for conversion
  Value range: [0, 28]

Email_Interaction:
  Current dtype: float64
  Null count:

Imputation, Currently not used

In [67]:
# Impute Price with category-specific median, create missing flag
df['missing_Price'] = df['Price'].isna().astype('int8')
df['Price'] = df.groupby('Category')['Price'].transform(
    lambda x: x.fillna(x.median())
)
# For categories with all-null Price, use global median
df['Price'] = df['Price'].fillna(df['Price'].median())
print(f"Price nulls after imputation: {df['Price'].isna().sum()}")

Price nulls after imputation: 0


In [68]:
# Impute Category with mode, or add "Unknown" category
df['missing_Category'] = df['Category'].isna().astype('int8')
mode_category = df['Category'].mode()[0] if not df['Category'].mode().empty else 0
df['Category'] = df['Category'].fillna(mode_category)
df['Category'] = df['Category'].astype('int8')
print(f"Category nulls after imputation: {df['Category'].isna().sum()}")

Category nulls after imputation: 0


In [69]:
# Items_In_Cart: fill with 0 (cart empty)
df['missing_Items_In_Cart'] = df['Items_In_Cart'].isna().astype('int8')
df['Items_In_Cart'] = df['Items_In_Cart'].fillna(0)
df['Items_In_Cart'] = df['Items_In_Cart'].astype('int8')

# Email_Interaction: fill with 0 (no interaction)
df['missing_Email_Interaction'] = df['Email_Interaction'].isna().astype('int8')
df['Email_Interaction'] = df['Email_Interaction'].fillna(0)
df['Email_Interaction'] = df['Email_Interaction'].astype('int8')

print(f"Items_In_Cart nulls: {df['Items_In_Cart'].isna().sum()}")
print(f"Email_Interaction nulls: {df['Email_Interaction'].isna().sum()}")

Items_In_Cart nulls: 0
Email_Interaction nulls: 0


In [70]:
# Time_of_Day: mode or "evening" (most common)
df['missing_Time_of_Day'] = df['Time_of_Day'].isna().astype('int8')
mode_time = df['Time_of_Day'].mode()[0] if not df['Time_of_Day'].mode().empty else 'evening'
df['Time_of_Day'] = df['Time_of_Day'].fillna(mode_time)

# Device_Type: mode or "Mobile"
df['missing_Device_Type'] = df['Device_Type'].isna().astype('int8')
mode_device = df['Device_Type'].mode()[0] if not df['Device_Type'].mode().empty else 'Mobile'
df['Device_Type'] = df['Device_Type'].fillna(mode_device)

print(f"Time_of_Day nulls: {df['Time_of_Day'].isna().sum()}")
print(f"Device_Type nulls: {df['Device_Type'].isna().sum()}")

Time_of_Day nulls: 0
Device_Type nulls: 0


In [9]:
# check if there are Items_In_Cart which are non zero for Purchase==0
nonzero_cart_no_purchase = df[(df['Purchase'] == 0) & (df['Items_In_Cart'] > 0)]
n_nonzero_cart_no_purchase = len(nonzero_cart_no_purchase)
print(f"Rows with Purchase==0 but Items_In_Cart>0: {n_nonzero_cart_no_purchase}")
if n_nonzero_cart_no_purchase > 0:
    display(nonzero_cart_no_purchase.head(10))

Rows with Purchase==0 but Items_In_Cart>0: 8170


,id,Age,Gender,Reviews_Read,Price,Discount,Category,Items_In_Cart,Time_of_Day,Email_Interaction,Device_Type,Payment_Method,Referral_Source,Socioeconomic_Status_Score,Engagement_Score,AB_Bucket,Price_Sine,PM_RS_Combo,Session_ID,Day,Campaign_Period,Purchase
0,1,NaN,1.0,3.0,592.975,22.0,1.0,6.0,afternoon,0.0,Mobile,Credit,NaN,7.26,1.856520,3.0,0.999047,Credit:Social_media,S0000003,59,False,0
2,3,22.0,0.0,3.0,218.360,2.0,1.0,4.0,evening,1.0,Mobile,Bank,Social_media,6.61,1.223445,0.0,-0.421646,Bank:Social_media,S0000007,16,False,0
4,5,35.0,1.0,1.0,495.088,13.0,0.0,2.0,evening,0.0,Mobile,Cash,Social_media,8.33,3.848580,2.0,0.695737,Cash:Social_media,S0000009,10,False,0
6,7,NaN,1.0,2.0,479.629,41.0,4.0,6.0,afternoon,1.0,Mobile,Bank,Search_engine,NaN,0.828184,4.0,0.998348,Bank:Search_engine,S0000011,26,True,0
7,8,19.0,1.0,4.0,NaN,12.0,3.0,14.0,morning,NaN,Desktop,Bank,Search_engine,1.62,1.486695,6.0,0.862095,Bank:Search_engine,S0000013,12,False,0
8,9,18.0,0.0,5.0,831.402,2.0,3.0,2.0,morning,0.0,Desktop,PayPal,Search_engine,6.23,1.376030,NaN,0.804534,PayPal:Search_engine,S0000014,47,True,0
9,10,22.0,1.0,4.0,234.562,14.0,1.0,2.0,morning,1.0,Mobile,Bank,Social_media,6.71,1.319739,2.0,0.448290,Bank:Social_media,S0000016,3,False,0
10,11,36.0,0.0,4.0,762.260,20.0,2.0,7.0,afternoon,1.0,Desktop,Bank,NaN,2.62,5.585908,5.0,-0.997970,Bank:Social_media,S0000019,58,False,0
11,12,27.0,0.0,2.0,431.600,4.0,4.0,2.0,morning,0.0,Desktop,NaN,Ads,0.00,1.717322,1.0,-0.914780,Cash:Ads,S0000022,45,True,0
15,16,37.0,0.0,5.0,732.919,35.0,4.0,6.0,afternoon,0.0,Desktop,Credit,Ads,7.60,1.107753,6.0,0.122672,Credit:Ads,S0000027,21,False,0
